# Module 13: Multi-Agent Systems & Advanced Patterns

## Overview

Modern AI applications increasingly rely on **multiple agents working together** rather than a single monolithic agent. Just as software engineering evolved from single-process programs to microservices, AI agents are evolving from single-agent systems to coordinated multi-agent architectures.

In this module, we go beyond single agents to explore:

1. **Multi-Agent Patterns**: Sequential, parallel, hierarchical, and debate architectures
2. **ReAct in Depth**: Full Thought-Action-Observation loop implemented from scratch
3. **Agent Memory**: Buffer, summary, and entity memory for persistent context
4. **Agent Planning**: Decomposing complex goals into executable sub-tasks
5. **Self-Reflection**: Agents that critique and improve their own output
6. **LangGraph**: Building agent workflows as state machines
7. **Debate Systems**: Multiple agents arguing opposing positions

**Note:** The `ai_trading_agent.ipynb` notebook in this repository demonstrates a 5-agent sequential chain using LangChain's `SequentialChain`. This module covers more modern and flexible patterns, including how to refactor that approach using LangGraph.

### What you'll learn

- How to architect multi-agent systems for different use cases
- How to implement ReAct reasoning from scratch (no framework)
- How agent memory works and why it matters for multi-turn interactions
- How planning and self-reflection make agents more capable
- How to use LangGraph to build robust agent workflows

## 1. Setup

In [ ]:
!pip install -q openai python-dotenv langchain langchain-openai langgraph

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv("/home/amir/source/.env")

import json
import re
import time
from typing import Any, TypedDict, Annotated
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL = "gpt-4o-mini"

print(f"OpenAI client initialized with model: {MODEL}")
print("All imports successful!")

In [ ]:
# Helper function used throughout the notebook
def chat(messages, model=MODEL, temperature=0.7, max_tokens=1024):
    """Simple wrapper around OpenAI chat completion."""
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

# Quick test
test_response = chat([{"role": "user", "content": "Say 'Module 13 is ready!' in exactly those words."}])
print(test_response)

## 2. Multi-Agent Patterns

There are four fundamental patterns for organizing multiple agents. Each suits different problem types.

### Pattern 1: Sequential

Agents run in order. The output of one agent becomes the input to the next.

```
  Input
    |
    v
+----------+     +----------+     +----------+
| Agent A  | --> | Agent B  | --> | Agent C  |
| Research |     | Writing  |     | Editing  |
+----------+     +----------+     +----------+
                                       |
                                       v
                                    Output
```

**When to use:** Pipeline tasks where each stage refines or transforms the previous output.  
**Example:** The trading agent in `ai_trading_agent.ipynb` -- data analyst -> sentiment analyst -> macro analyst -> strategist -> risk manager.

### Pattern 2: Parallel

Independent agents run simultaneously on the same input. Results are combined.

```
              Input
            /   |   \
           v    v    v
       +------+------+------+
       |  A   |  B   |  C   |
       | Fact | Style| Tone |
       |Check |Check |Check |
       +------+------+------+
           \    |    /
            v   v   v
         +-----------+
         | Combiner  |
         +-----------+
              |
              v
           Output
```

**When to use:** Tasks that can be broken into independent sub-problems (e.g., multi-criteria evaluation).  
**Example:** Code review where separate agents check style, security, and performance.

### Pattern 3: Hierarchical

A manager agent delegates tasks to worker agents and synthesizes their results.

```
              +---------+
              | Manager |
              +---------+
             /     |     \
            v      v      v
       +------+ +------+ +------+
       |Worker| |Worker| |Worker|
       |  A   | |  B   | |  C   |
       +------+ +------+ +------+
            \      |      /
             v     v     v
              +---------+
              | Manager |
              |(synth.) |
              +---------+
```

**When to use:** Complex tasks requiring dynamic delegation and coordination.  
**Example:** A project manager agent that assigns research, writing, and review tasks to specialist agents.

### Pattern 4: Debate

Agents argue opposing positions. A judge agent evaluates and decides.

```
       +----------+     +----------+
       | Pro Agent | <-> | Con Agent|
       +----------+     +----------+
            |                 |
            v                 v
       +--------------------------+
       |       Judge Agent        |
       | (evaluates both sides)   |
       +--------------------------+
                   |
                   v
              Final Verdict
```

**When to use:** Decisions requiring balanced analysis, reducing bias, or adversarial testing.  
**Example:** Evaluating whether a business proposal is viable by having pro and con agents debate.

### Implementing the Sequential Pattern

In [ ]:
def run_sequential_agents(topic):
    """Sequential pattern: researcher -> writer -> editor."""
    print(f"=== Sequential Pipeline for: '{topic}' ===\n")
    
    # Agent 1: Researcher
    print("[Agent 1: Researcher]")
    research = chat([
        {"role": "system", "content": "You are a research analyst. Provide 3-4 key facts about the given topic. Be concise and factual."},
        {"role": "user", "content": f"Research this topic: {topic}"}
    ])
    print(research)
    print()
    
    # Agent 2: Writer (uses research output)
    print("[Agent 2: Writer]")
    draft = chat([
        {"role": "system", "content": "You are a skilled writer. Given research notes, write a short 2-paragraph article. Make it engaging and informative."},
        {"role": "user", "content": f"Write an article based on these research notes:\n\n{research}"}
    ])
    print(draft)
    print()
    
    # Agent 3: Editor (uses writer output)
    print("[Agent 3: Editor]")
    final = chat([
        {"role": "system", "content": "You are a meticulous editor. Review the article for clarity, grammar, and engagement. Provide the improved version."},
        {"role": "user", "content": f"Edit and improve this article:\n\n{draft}"}
    ])
    print(final)
    
    return {"research": research, "draft": draft, "final": final}

result = run_sequential_agents("The impact of transformers on natural language processing")

### Implementing the Parallel Pattern

In [ ]:
def run_parallel_agents(text):
    """Parallel pattern: multiple reviewers analyze text independently, then combine."""
    print(f"=== Parallel Review Pipeline ===\n")
    print(f"Text to review: '{text[:80]}...'\n")
    
    reviewers = {
        "Fact Checker": "You are a fact checker. Evaluate the factual accuracy of the given text. List any claims that need verification or correction. Be concise.",
        "Style Reviewer": "You are a writing style expert. Evaluate the clarity, readability, and engagement of the text. Suggest specific improvements. Be concise.",
        "Tone Analyst": "You are a tone and audience analyst. Evaluate whether the tone is appropriate for a general audience. Note any issues. Be concise."
    }
    
    # Run all reviewers (in practice these could be concurrent; here sequential for simplicity)
    reviews = {}
    for name, system_prompt in reviewers.items():
        print(f"[{name}]")
        review = chat([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Review this text:\n\n{text}"}
        ], max_tokens=300)
        reviews[name] = review
        print(review)
        print()
    
    # Combiner agent synthesizes all reviews
    print("[Combiner Agent]")
    combined_reviews = "\n\n".join([f"### {name}:\n{review}" for name, review in reviews.items()])
    synthesis = chat([
        {"role": "system", "content": "You are an editorial coordinator. Given multiple reviewer assessments, synthesize them into a single prioritized list of 3-5 action items for the author."},
        {"role": "user", "content": f"Here are the reviews:\n\n{combined_reviews}"}
    ], max_tokens=400)
    print(synthesis)
    
    return {"reviews": reviews, "synthesis": synthesis}

sample_text = """Quantum computing leverages quantum mechanical phenomena like superposition 
and entanglement to process information in fundamentally new ways. Unlike classical 
computers that use bits (0 or 1), quantum computers use qubits that can exist in 
multiple states simultaneously. This allows them to solve certain problems exponentially 
faster than any classical computer. Google achieved quantum supremacy in 2019 when their 
Sycamore processor performed a calculation in 200 seconds that would take the world's 
fastest supercomputer 10,000 years."""

result = run_parallel_agents(sample_text)

## 3. ReAct in Depth

ReAct (Reasoning + Acting) is a foundational agent pattern where the LLM alternates between:

1. **Thought**: The agent reasons about what to do next
2. **Action**: The agent selects a tool and provides input
3. **Observation**: The tool returns a result
4. **Repeat** until the agent has a final answer

```
User Question
     |
     v
+--------------------+
| Thought: I need to |<---+
| search for X...    |    |
+--------------------+    |
     |                    |
     v                    |
+--------------------+    |
| Action: search     |    |
| Action Input: X    |    |
+--------------------+    |
     |                    |
     v                    |
+--------------------+    |
| Observation: ...   |----+
| (tool result)      |    |
+--------------------+    |
     |                    |
     v                    |
  Has enough    No -------+
  info?
     | Yes
     v
Final Answer
```

Let's implement a full ReAct agent **from scratch** -- no framework.

In [ ]:
# Define simple tools for our ReAct agent

def calculator(expression: str) -> str:
    """Evaluate a mathematical expression."""
    try:
        # Only allow safe math operations
        allowed_names = {"abs": abs, "round": round, "min": min, "max": max, "pow": pow}
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return str(result)
    except Exception as e:
        return f"Error: {e}"

def lookup(term: str) -> str:
    """Look up a fact from a small knowledge base."""
    knowledge = {
        "python": "Python is a programming language created by Guido van Rossum, first released in 1991. It emphasizes readability and simplicity.",
        "transformer": "The Transformer architecture was introduced in the 2017 paper 'Attention Is All You Need' by Vaswani et al. It uses self-attention mechanisms.",
        "gpt": "GPT (Generative Pre-trained Transformer) is a family of language models by OpenAI. GPT-3 has 175 billion parameters.",
        "bert": "BERT (Bidirectional Encoder Representations from Transformers) was released by Google in 2018. It uses masked language modeling for pre-training.",
        "attention": "Attention mechanism allows models to focus on relevant parts of the input. Self-attention computes relationships between all positions in a sequence.",
        "rag": "RAG (Retrieval-Augmented Generation) combines a retriever that finds relevant documents with a generator that produces answers based on those documents.",
        "langchain": "LangChain is a framework for developing applications powered by language models. It provides tools for chains, agents, and memory.",
        "embedding": "An embedding is a dense vector representation of data (text, images) in a continuous vector space where similar items are close together."
    }
    term_lower = term.lower().strip()
    for key, value in knowledge.items():
        if key in term_lower or term_lower in key:
            return value
    return f"No information found for '{term}'. Available topics: {', '.join(knowledge.keys())}"

def string_length(text: str) -> str:
    """Count the number of characters in a string."""
    return str(len(text))

# Tool registry
TOOLS = {
    "calculator": {"fn": calculator, "desc": "Evaluate a math expression. Input: a mathematical expression like '2 + 3 * 4'"},
    "lookup": {"fn": lookup, "desc": "Look up information about a topic. Input: a topic name like 'python' or 'transformer'"},
    "string_length": {"fn": string_length, "desc": "Count characters in a string. Input: any text string"}
}

print("Tools registered:")
for name, tool in TOOLS.items():
    print(f"  - {name}: {tool['desc']}")

In [ ]:
# ReAct agent implementation from scratch

REACT_SYSTEM_PROMPT = """You are a helpful assistant that answers questions by reasoning step-by-step and using tools.

You have access to the following tools:
{tool_descriptions}

To use a tool, respond with EXACTLY this format:
Thought: <your reasoning about what to do next>
Action: <tool_name>
Action Input: <input to the tool>

After receiving an observation (tool result), continue reasoning.

When you have enough information to answer, respond with:
Thought: I now have enough information to answer.
Final Answer: <your complete answer>

IMPORTANT:
- Always start with a Thought
- Use exactly one tool per step
- Do not make up information; use tools to find facts
- Always end with 'Final Answer:' when done
"""

def parse_react_output(text):
    """Parse the LLM output for Thought, Action, Action Input, or Final Answer."""
    result = {"thought": None, "action": None, "action_input": None, "final_answer": None}
    
    # Extract thought
    thought_match = re.search(r"Thought:\s*(.+?)(?=\nAction:|\nFinal Answer:|$)", text, re.DOTALL)
    if thought_match:
        result["thought"] = thought_match.group(1).strip()
    
    # Check for final answer
    final_match = re.search(r"Final Answer:\s*(.+)", text, re.DOTALL)
    if final_match:
        result["final_answer"] = final_match.group(1).strip()
        return result
    
    # Extract action and action input
    action_match = re.search(r"Action:\s*(.+)", text)
    if action_match:
        result["action"] = action_match.group(1).strip()
    
    input_match = re.search(r"Action Input:\s*(.+)", text)
    if input_match:
        result["action_input"] = input_match.group(1).strip()
    
    return result


def react_agent(question, max_steps=5, verbose=True):
    """Run a ReAct agent loop from scratch."""
    # Build tool descriptions for the system prompt
    tool_desc = "\n".join([f"- {name}: {info['desc']}" for name, info in TOOLS.items()])
    system_prompt = REACT_SYSTEM_PROMPT.format(tool_descriptions=tool_desc)
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    
    if verbose:
        print(f"Question: {question}")
        print("=" * 60)
    
    for step in range(max_steps):
        # Get LLM response
        response = chat(messages, temperature=0.0, max_tokens=500)
        
        if verbose:
            print(f"\n--- Step {step + 1} ---")
            print(response)
        
        # Parse the response
        parsed = parse_react_output(response)
        
        # Check if we have a final answer
        if parsed["final_answer"]:
            if verbose:
                print("\n" + "=" * 60)
                print(f"FINAL ANSWER: {parsed['final_answer']}")
            return parsed["final_answer"]
        
        # Execute the action
        if parsed["action"] and parsed["action"] in TOOLS:
            tool_fn = TOOLS[parsed["action"]]["fn"]
            observation = tool_fn(parsed["action_input"] or "")
            
            if verbose:
                print(f"\nObservation: {observation}")
            
            # Add the assistant response and observation to messages
            messages.append({"role": "assistant", "content": response})
            messages.append({"role": "user", "content": f"Observation: {observation}"})
        elif parsed["action"]:
            # Unknown tool
            error_msg = f"Unknown tool '{parsed['action']}'. Available tools: {', '.join(TOOLS.keys())}"
            messages.append({"role": "assistant", "content": response})
            messages.append({"role": "user", "content": f"Observation: {error_msg}"})
            if verbose:
                print(f"\nObservation: {error_msg}")
        else:
            # Could not parse action -- ask the model to try again
            messages.append({"role": "assistant", "content": response})
            messages.append({"role": "user", "content": "Please respond using the exact format: Thought/Action/Action Input or Thought/Final Answer."})
    
    return "Max steps reached without a final answer."

print("ReAct agent ready!")

In [ ]:
# Test the ReAct agent with a multi-step question
answer = react_agent(
    "What year was the Transformer architecture introduced, and what is 2024 minus that year?"
)

In [ ]:
# Another example: multi-tool usage
answer = react_agent(
    "Look up what GPT is, then calculate how many characters are in the description you find."
)

## 4. Agent Memory

Without memory, every agent interaction starts from scratch. Memory gives agents the ability to:

- **Recall** previous conversation context
- **Track** entities and facts mentioned over time
- **Maintain** coherent multi-turn conversations

### Types of Agent Memory

| Memory Type | How it Works | Pros | Cons |
|-------------|-------------|------|------|
| **Buffer** | Store all messages verbatim | Full context preserved | Token limit grows linearly |
| **Summary** | Periodically summarize old messages | Bounded token usage | Lossy -- details may be lost |
| **Entity** | Extract and track entities | Structured knowledge | Requires entity extraction |

```
Buffer Memory:       [msg1, msg2, msg3, msg4, msg5, ...]
                     (keeps everything -- tokens grow fast)

Summary Memory:      [summary_of_msg1-3, msg4, msg5, ...]
                     (compresses old messages -- bounded tokens)

Entity Memory:       {"Alice": "software engineer, lives in SF",
                      "Project X": "ML pipeline, deadline March"}
                     (structured entity tracking)
```

In [ ]:
# Implementation: Buffer Memory

class BufferMemory:
    """Simple buffer memory that stores all conversation messages."""
    
    def __init__(self, system_prompt="You are a helpful assistant."):
        self.system_prompt = system_prompt
        self.messages = [{"role": "system", "content": system_prompt}]
    
    def add_user_message(self, content):
        self.messages.append({"role": "user", "content": content})
    
    def add_assistant_message(self, content):
        self.messages.append({"role": "assistant", "content": content})
    
    def get_messages(self):
        return self.messages.copy()
    
    def token_estimate(self):
        """Rough estimate of token count (4 chars per token heuristic)."""
        total_chars = sum(len(m["content"]) for m in self.messages)
        return total_chars // 4
    
    def __repr__(self):
        return f"BufferMemory(messages={len(self.messages)}, ~{self.token_estimate()} tokens)"


# Implementation: Summary Memory

class SummaryMemory:
    """Memory that summarizes old messages to stay within token limits."""
    
    def __init__(self, system_prompt="You are a helpful assistant.", max_messages=6):
        self.system_prompt = system_prompt
        self.summary = ""
        self.recent_messages = []
        self.max_messages = max_messages  # keep this many recent messages before summarizing
    
    def add_user_message(self, content):
        self.recent_messages.append({"role": "user", "content": content})
        self._maybe_summarize()
    
    def add_assistant_message(self, content):
        self.recent_messages.append({"role": "assistant", "content": content})
        self._maybe_summarize()
    
    def _maybe_summarize(self):
        """If we have too many messages, summarize the older ones."""
        if len(self.recent_messages) > self.max_messages:
            # Take the older half of messages and summarize
            to_summarize = self.recent_messages[:self.max_messages // 2]
            self.recent_messages = self.recent_messages[self.max_messages // 2:]
            
            # Build the text to summarize
            convo_text = "\n".join([f"{m['role']}: {m['content']}" for m in to_summarize])
            prev_summary = f"Previous summary: {self.summary}\n\n" if self.summary else ""
            
            self.summary = chat([
                {"role": "system", "content": "Summarize this conversation concisely, preserving key facts, names, and decisions. Output only the summary."},
                {"role": "user", "content": f"{prev_summary}New messages to incorporate:\n{convo_text}"}
            ], max_tokens=200)
    
    def get_messages(self):
        messages = [{"role": "system", "content": self.system_prompt}]
        if self.summary:
            messages.append({"role": "system", "content": f"Summary of earlier conversation: {self.summary}"})
        messages.extend(self.recent_messages)
        return messages
    
    def __repr__(self):
        return f"SummaryMemory(recent={len(self.recent_messages)}, has_summary={bool(self.summary)})"


print("Memory classes defined!")
print(f"BufferMemory: stores all messages verbatim")
print(f"SummaryMemory: compresses old messages into summaries")

In [ ]:
# Demonstrate buffer memory across multiple turns

def chat_with_memory(memory, user_message):
    """Send a message using the given memory, store the exchange."""
    memory.add_user_message(user_message)
    response = chat(memory.get_messages(), max_tokens=300)
    memory.add_assistant_message(response)
    return response

# Demo: multi-turn conversation with buffer memory
print("=== Buffer Memory Demo ===")
print()

buf_mem = BufferMemory(system_prompt="You are a helpful AI tutor. Remember everything the student tells you.")

turns = [
    "Hi! My name is Alex and I'm learning about machine learning.",
    "I'm particularly interested in NLP. What should I learn first?",
    "I already know Python and basic linear algebra. Does that help?",
    "What's my name and what am I interested in?"  # Tests memory!
]

for i, msg in enumerate(turns, 1):
    print(f"--- Turn {i} ---")
    print(f"User: {msg}")
    response = chat_with_memory(buf_mem, msg)
    print(f"Assistant: {response}")
    print(f"[Memory state: {buf_mem}]")
    print()

### Exercise 1: Agent with Conversation Memory

Build a `MemoryAgent` class that:
1. Uses buffer memory to remember all previous exchanges
2. Has a `chat(message)` method that returns the assistant's response
3. Has a `memory_stats()` method that returns conversation statistics
4. Can answer questions about earlier parts of the conversation

**Requirements:**
- The agent should have a configurable system prompt
- It should track the number of turns
- The `memory_stats()` method should return a dict with `turns`, `total_messages`, and `estimated_tokens`

In [ ]:
# Exercise 1: Implement a MemoryAgent

class MemoryAgent:
    """An agent with conversation memory across interactions."""
    
    def __init__(self, system_prompt="You are a helpful assistant with perfect memory."):
        # TODO: Initialize the agent with a system prompt and buffer memory
        self.system_prompt = system_prompt
        self.messages = None  # TODO: initialize message list with system prompt
        self.turns = None  # TODO: initialize turn counter
    
    def chat(self, message):
        """
        Send a message to the agent and get a response.
        The agent remembers all previous exchanges.
        
        Args:
            message: The user's message
        Returns:
            The assistant's response string
        """
        # TODO: Add the user message to memory
        # TODO: Call the LLM with the full message history
        # TODO: Add the assistant response to memory
        # TODO: Increment turn counter
        # TODO: Return the response
        return None
    
    def memory_stats(self):
        """
        Return statistics about the conversation memory.
        
        Returns:
            dict with keys: 'turns', 'total_messages', 'estimated_tokens'
        """
        # TODO: Calculate and return memory statistics
        return None

# Test (will fail until you complete the TODOs)
# agent = MemoryAgent(system_prompt="You are a friendly AI tutor.")
# print(agent.chat("My name is Jordan. I'm studying transformers."))
# print(agent.chat("What's my name?"))
# print(agent.memory_stats())

### Solution

In [ ]:
# Solution: MemoryAgent with full conversation memory

class MemoryAgent:
    """An agent with conversation memory across interactions."""
    
    def __init__(self, system_prompt="You are a helpful assistant with perfect memory."):
        self.system_prompt = system_prompt
        self.messages = [{"role": "system", "content": system_prompt}]
        self.turns = 0
    
    def chat(self, message):
        """Send a message and get a response. Memory persists across calls."""
        # Add user message to memory
        self.messages.append({"role": "user", "content": message})
        
        # Call the LLM with full history
        response = chat(self.messages, max_tokens=400)
        
        # Store response in memory
        self.messages.append({"role": "assistant", "content": response})
        
        # Increment turn counter
        self.turns += 1
        
        return response
    
    def memory_stats(self):
        """Return conversation memory statistics."""
        total_chars = sum(len(m["content"]) for m in self.messages)
        return {
            "turns": self.turns,
            "total_messages": len(self.messages),
            "estimated_tokens": total_chars // 4
        }


# 5-turn demo showing memory retention
print("=== MemoryAgent 5-Turn Demo ===")
print()

agent = MemoryAgent(system_prompt="You are a friendly AI tutor who remembers everything about the student.")

conversation = [
    "Hi! My name is Jordan and I'm a data scientist at TechCorp.",
    "I want to learn about multi-agent systems. I already know LangChain basics.",
    "My favorite part of AI is the reasoning capabilities of LLMs.",
    "Can you suggest a project that combines what I know with multi-agent systems?",
    "Quick quiz: what's my name, where do I work, and what's my favorite part of AI?"
]

for i, msg in enumerate(conversation, 1):
    print(f"--- Turn {i} ---")
    print(f"User: {msg}")
    response = agent.chat(msg)
    print(f"Agent: {response}")
    stats = agent.memory_stats()
    print(f"[Stats: {stats}]")
    print()

## 5. Agent Planning

Complex tasks require **planning** -- breaking a big goal into smaller, manageable sub-tasks.

The planning pattern:

```
Complex Goal
     |
     v
+-------------------+
| Planner Agent     |
| (decompose goal)  |
+-------------------+
     |
     v
Step 1 -> Execute -> Result 1
Step 2 -> Execute -> Result 2
Step 3 -> Execute -> Result 3
     |
     v
+-------------------+
| Synthesizer Agent |
| (combine results) |
+-------------------+
     |
     v
Final Output
```

This is powerful because:
- Each sub-task is simpler and more likely to succeed
- You can retry individual steps without redoing everything
- The plan itself can be inspected and modified

In [ ]:
# Planning agent implementation

def planning_agent(goal, verbose=True):
    """
    Agent that decomposes a complex goal into sub-tasks, executes each, 
    and synthesizes the results.
    """
    if verbose:
        print(f"=== Planning Agent ===")
        print(f"Goal: {goal}")
        print("=" * 60)
    
    # Step 1: Plan -- decompose the goal into sub-tasks
    if verbose:
        print("\n[Phase 1: PLANNING]")
    
    plan_response = chat([
        {"role": "system", "content": """You are a planning agent. Given a complex goal, break it down into 3-5 concrete sub-tasks.
Output your plan as a JSON array of strings, each being one sub-task.
Example: ["Research topic X", "Outline key points", "Write first draft", "Review and edit"]
Output ONLY the JSON array, nothing else.""" },
        {"role": "user", "content": f"Break this goal into sub-tasks: {goal}"}
    ], temperature=0.3, max_tokens=300)
    
    # Parse the plan
    try:
        # Find JSON array in response
        json_match = re.search(r'\[.*\]', plan_response, re.DOTALL)
        if json_match:
            steps = json.loads(json_match.group())
        else:
            steps = json.loads(plan_response)
    except json.JSONDecodeError:
        # Fallback: split by newlines
        steps = [line.strip().lstrip('0123456789.-) ') for line in plan_response.strip().split('\n') if line.strip()]
    
    if verbose:
        print(f"Plan ({len(steps)} steps):")
        for i, step in enumerate(steps, 1):
            print(f"  {i}. {step}")
    
    # Step 2: Execute each sub-task
    if verbose:
        print("\n[Phase 2: EXECUTION]")
    
    results = []
    for i, step in enumerate(steps, 1):
        if verbose:
            print(f"\n  Executing step {i}: {step}")
        
        # Context from previous steps
        context = ""
        if results:
            context = "Previous step results:\n" + "\n".join(
                [f"Step {j+1}: {r[:200]}..." if len(r) > 200 else f"Step {j+1}: {r}" 
                 for j, r in enumerate(results)]
            ) + "\n\n"
        
        step_result = chat([
            {"role": "system", "content": f"You are executing one step of a larger plan. The overall goal is: {goal}. Complete only the current step thoroughly and concisely."},
            {"role": "user", "content": f"{context}Current step to execute: {step}"}
        ], max_tokens=500)
        
        results.append(step_result)
        if verbose:
            print(f"  Result: {step_result[:200]}..." if len(step_result) > 200 else f"  Result: {step_result}")
    
    # Step 3: Synthesize all results
    if verbose:
        print("\n[Phase 3: SYNTHESIS]")
    
    all_results = "\n\n".join([f"### Step {i+1}: {steps[i]}\n{result}" for i, result in enumerate(results)])
    
    final = chat([
        {"role": "system", "content": "You are a synthesis agent. Combine the results from multiple completed sub-tasks into a coherent, well-organized final output. Maintain quality and flow."},
        {"role": "user", "content": f"Goal: {goal}\n\nCompleted sub-task results:\n\n{all_results}\n\nSynthesize these into a final, polished output."}
    ], max_tokens=800)
    
    if verbose:
        print(f"\nFinal Output:\n{final}")
    
    return {"plan": steps, "step_results": results, "final": final}

print("Planning agent ready!")

In [ ]:
# Demo: planning agent in action
result = planning_agent("Explain the differences between CNNs and Transformers for a beginner audience")

### Exercise 2: Planning Agent for Research Reports

Build a planning agent that decomposes the task of "writing a research report" into sub-tasks, executes each sub-task, and combines the results into a final report.

**Requirements:**
- `research_report_agent(topic)` function
- Must generate a plan with at least 4 steps
- Each step should be executed with context from previous steps
- Final output should be a coherent report
- Print each phase (planning, execution, synthesis)

In [ ]:
# Exercise 2: Planning agent for research reports

def research_report_agent(topic):
    """
    Agent that plans, researches, and writes a short report on the given topic.
    
    Args:
        topic: The research topic to write about
    Returns:
        dict with 'plan', 'step_results', and 'report' keys
    """
    print(f"=== Research Report Agent ===")
    print(f"Topic: {topic}")
    print("=" * 60)
    
    # Phase 1: Create a research plan
    # TODO: Use the LLM to generate a plan with at least 4 steps
    # The plan should cover: background research, key concepts,
    # analysis/comparison, and writing/structuring
    plan = None  # Should be a list of step strings
    
    # Phase 2: Execute each step
    # TODO: Loop through the plan, execute each step with context
    # from previous steps, collect results
    step_results = None  # Should be a list of result strings
    
    # Phase 3: Synthesize into final report
    # TODO: Combine all step results into a coherent report
    report = None
    
    return {"plan": plan, "step_results": step_results, "report": report}

# Test (will fail until you complete the TODOs)
# result = research_report_agent("The evolution of attention mechanisms in deep learning")

### Solution

In [ ]:
# Solution: Planning agent for research reports

def research_report_agent(topic):
    """
    Agent that plans, researches, and writes a short report on the given topic.
    """
    print(f"=== Research Report Agent ===")
    print(f"Topic: {topic}")
    print("=" * 60)
    
    # Phase 1: Create a research plan
    print("\n[Phase 1: PLANNING]")
    plan_response = chat([
        {"role": "system", "content": """You are a research planner. Given a topic, create a structured plan for writing a short research report.
The plan must have exactly 5 steps covering:
1. Background and historical context
2. Key concepts and terminology
3. Current state of the art
4. Challenges and open problems
5. Future directions and conclusion

Output ONLY a JSON array of 5 strings, each describing one research step.
Example: ["Research the historical background of X", "Define key concepts", ...]"""},
        {"role": "user", "content": f"Create a research plan for: {topic}"}
    ], temperature=0.3, max_tokens=400)
    
    try:
        json_match = re.search(r'\[.*\]', plan_response, re.DOTALL)
        if json_match:
            plan = json.loads(json_match.group())
        else:
            plan = json.loads(plan_response)
    except json.JSONDecodeError:
        plan = [line.strip().lstrip('0123456789.-) ') for line in plan_response.strip().split('\n') if line.strip()]
    
    print(f"Plan ({len(plan)} steps):")
    for i, step in enumerate(plan, 1):
        print(f"  {i}. {step}")
    
    # Phase 2: Execute each step with cumulative context
    print("\n[Phase 2: EXECUTION]")
    step_results = []
    
    for i, step in enumerate(plan):
        print(f"\n  Executing step {i+1}/{len(plan)}: {step}")
        
        # Build context from previous results
        context = ""
        if step_results:
            context = "Context from completed research steps:\n"
            for j, prev_result in enumerate(step_results):
                truncated = prev_result[:300] + "..." if len(prev_result) > 300 else prev_result
                context += f"\nStep {j+1} result: {truncated}\n"
            context += "\n"
        
        result = chat([
            {"role": "system", "content": f"""You are a research writer working on a report about: {topic}.
Complete only the current step. Write 2-3 detailed paragraphs.
Use an academic but accessible tone."""},
            {"role": "user", "content": f"{context}Current step: {step}"}
        ], max_tokens=600)
        
        step_results.append(result)
        preview = result[:150] + "..." if len(result) > 150 else result
        print(f"  Done. Preview: {preview}")
    
    # Phase 3: Synthesize into a cohesive report
    print("\n[Phase 3: SYNTHESIS]")
    
    all_sections = "\n\n".join([
        f"### Section {i+1}: {plan[i]}\n{result}" 
        for i, result in enumerate(step_results)
    ])
    
    report = chat([
        {"role": "system", "content": """You are a senior editor. Combine the following research sections into a polished, 
cohesive report. Add smooth transitions between sections, a brief introduction, and a conclusion.
Use markdown formatting with headers. Keep the total length reasonable (800-1200 words)."""},
        {"role": "user", "content": f"Topic: {topic}\n\nResearch sections to combine:\n\n{all_sections}"}
    ], max_tokens=2000)
    
    print(f"\n{'=' * 60}")
    print("FINAL REPORT:")
    print("=" * 60)
    print(report)
    
    return {"plan": plan, "step_results": step_results, "report": report}

# Run the research report agent
result = research_report_agent("The evolution of attention mechanisms in deep learning")

## 6. Self-Reflection

One of the most powerful patterns in agentic AI is **self-reflection** -- agents that critique and improve their own output.

The pattern is simple but effective:

```
Prompt
  |
  v
+----------+     +----------+     +----------+
| Generate | --> | Critique | --> | Revise   |
| (draft)  |     | (review) |     | (improve)|
+----------+     +----------+     +----------+
                      |                 |
                      +---- repeat -----+
```

This mimics how humans write: draft, review, revise. Each iteration typically improves quality.

In [ ]:
def self_reflection_agent(task, iterations=3, verbose=True):
    """
    Agent that generates output, critiques it, and iteratively improves it.
    
    Args:
        task: The task to complete
        iterations: Number of generate-critique-revise cycles
        verbose: Whether to print intermediate steps
    Returns:
        dict with 'drafts', 'critiques', and 'final' keys
    """
    if verbose:
        print(f"=== Self-Reflection Agent ({iterations} iterations) ===")
        print(f"Task: {task}")
        print("=" * 60)
    
    drafts = []
    critiques = []
    
    # Initial generation
    if verbose:
        print("\n[Iteration 1: Initial Draft]")
    
    current_draft = chat([
        {"role": "system", "content": "You are a skilled writer. Complete the task to the best of your ability."},
        {"role": "user", "content": task}
    ], max_tokens=600)
    
    drafts.append(current_draft)
    if verbose:
        print(f"Draft 1:\n{current_draft}\n")
    
    # Iterative critique and revision
    for i in range(1, iterations):
        # Critique
        if verbose:
            print(f"[Iteration {i+1}: Critique]")
        
        critique = chat([
            {"role": "system", "content": """You are a critical reviewer. Analyze the given text and provide:
1. Three specific strengths
2. Three specific weaknesses or areas for improvement
3. A concrete suggestion for each weakness
Be constructive and specific."""},
            {"role": "user", "content": f"Original task: {task}\n\nText to critique:\n{current_draft}"}
        ], max_tokens=500)
        
        critiques.append(critique)
        if verbose:
            print(f"Critique:\n{critique}\n")
        
        # Revise
        if verbose:
            print(f"[Iteration {i+1}: Revision]")
        
        current_draft = chat([
            {"role": "system", "content": "You are a skilled writer revising your work based on feedback. Address all the critique points while maintaining what works well."},
            {"role": "user", "content": f"Original task: {task}\n\nYour previous draft:\n{current_draft}\n\nCritique to address:\n{critique}\n\nPlease write an improved version."}
        ], max_tokens=700)
        
        drafts.append(current_draft)
        if verbose:
            print(f"Revised Draft {i+1}:\n{current_draft}\n")
    
    if verbose:
        print("=" * 60)
        print(f"Completed {iterations} iterations of self-reflection.")
        print(f"Drafts: {len(drafts)}, Critiques: {len(critiques)}")
    
    return {"drafts": drafts, "critiques": critiques, "final": current_draft}

print("Self-reflection agent ready!")

In [ ]:
# Demo: watch the agent improve its output over iterations
result = self_reflection_agent(
    "Write a concise explanation of how attention mechanisms work in transformers, suitable for someone who knows basic neural networks but not transformers.",
    iterations=3
)

In [ ]:
# Compare first draft vs final draft
print("=== FIRST DRAFT ===")
print(result["drafts"][0])
print()
print("=" * 60)
print()
print("=== FINAL DRAFT (after self-reflection) ===")
print(result["final"])
print()
print(f"First draft length: {len(result['drafts'][0])} chars")
print(f"Final draft length: {len(result['final'])} chars")

## 7. LangGraph Introduction

LangGraph represents agent workflows as **state machines** -- directed graphs where:

- **Nodes** are functions (agents, tools, or processing steps)
- **Edges** define transitions between nodes
- **State** is a shared data structure passed between nodes
- **Conditional edges** allow dynamic routing based on state

```
Traditional Chain:    A -> B -> C  (linear, rigid)

LangGraph:            A -> B -> C  (if state.ok)
                           |-> D  (if state.needs_review)
                           |-> A  (if state.retry)
```

This is much more flexible than LangChain's `SequentialChain` because:
1. Nodes can loop (retry failed steps)
2. Routing can depend on runtime state
3. The workflow structure is explicit and inspectable

In [ ]:
from langgraph.graph import StateGraph, START, END

# Define the state that will be passed between nodes
class ArticleState(TypedDict):
    topic: str
    research: str
    draft: str
    critique: str
    final_article: str
    revision_count: int
    needs_revision: bool

# Define node functions -- each takes state and returns updated state

def researcher_node(state: ArticleState) -> dict:
    """Research the topic and gather key information."""
    print("[Node: Researcher]")
    research = chat([
        {"role": "system", "content": "You are a research analyst. Provide 4-5 key facts and insights about the topic. Be thorough but concise."},
        {"role": "user", "content": f"Research this topic: {state['topic']}"}
    ], max_tokens=500)
    print(f"  Research complete ({len(research)} chars)")
    return {"research": research}

def writer_node(state: ArticleState) -> dict:
    """Write or revise the article based on research and any critique."""
    print("[Node: Writer]")
    
    if state.get("critique") and state.get("draft"):
        # Revision mode
        prompt = f"""Revise this article based on the critique.

Original article:
{state['draft']}

Critique:
{state['critique']}

Write an improved version addressing all feedback."""
    else:
        # Initial draft mode
        prompt = f"""Write a short, engaging article (3-4 paragraphs) based on this research:

{state['research']}"""
    
    draft = chat([
        {"role": "system", "content": "You are a skilled writer. Write clear, engaging content."},
        {"role": "user", "content": prompt}
    ], max_tokens=600)
    
    revision_count = state.get("revision_count", 0) + 1
    print(f"  Draft written (revision {revision_count}, {len(draft)} chars)")
    return {"draft": draft, "revision_count": revision_count}

def critic_node(state: ArticleState) -> dict:
    """Critique the article and decide if revision is needed."""
    print("[Node: Critic]")
    critique = chat([
        {"role": "system", "content": """You are a critical editor. Review the article and:
1. Rate it 1-10 for quality
2. List specific improvements needed (if any)
3. Start your response with 'SCORE: X/10'

If the score is 8 or above, the article is ready for publication."""},
        {"role": "user", "content": f"Article to review:\n\n{state['draft']}"}
    ], max_tokens=400)
    
    # Parse score to determine if revision is needed
    score_match = re.search(r'SCORE:\s*(\d+)', critique)
    score = int(score_match.group(1)) if score_match else 5
    needs_revision = score < 8 and state.get("revision_count", 0) < 3
    
    print(f"  Score: {score}/10, Needs revision: {needs_revision}")
    return {"critique": critique, "needs_revision": needs_revision}

def finalizer_node(state: ArticleState) -> dict:
    """Prepare the final article."""
    print("[Node: Finalizer]")
    print(f"  Article finalized after {state.get('revision_count', 0)} revision(s)")
    return {"final_article": state["draft"]}


# Conditional routing function
def should_revise(state: ArticleState) -> str:
    """Determine next node based on whether revision is needed."""
    if state.get("needs_revision", False):
        return "writer"  # Go back to writer for revision
    return "finalizer"   # Move to finalization


print("LangGraph nodes defined!")

In [ ]:
# Build the LangGraph workflow

workflow = StateGraph(ArticleState)

# Add nodes
workflow.add_node("researcher", researcher_node)
workflow.add_node("writer", writer_node)
workflow.add_node("critic", critic_node)
workflow.add_node("finalizer", finalizer_node)

# Add edges
workflow.add_edge(START, "researcher")       # Start with research
workflow.add_edge("researcher", "writer")     # Research -> Write
workflow.add_edge("writer", "critic")          # Write -> Critique

# Conditional edge: critic decides whether to revise or finalize
workflow.add_conditional_edges(
    "critic",
    should_revise,
    {
        "writer": "writer",       # Loop back for revision
        "finalizer": "finalizer"  # Move to finalization
    }
)

workflow.add_edge("finalizer", END)  # Finalizer -> End

# Compile the graph
app = workflow.compile()

print("LangGraph workflow compiled!")
print("")
print("Graph structure:")
print("  START -> researcher -> writer -> critic")
print("                           ^         |")
print("                           |    (conditional)")
print("                           +-- needs_revision --+")
print("                                     |")
print("                               finalizer -> END")

In [ ]:
# Run the LangGraph workflow
print("=== Running LangGraph Article Pipeline ===")
print("=" * 60)

initial_state = {
    "topic": "How retrieval-augmented generation (RAG) is changing enterprise AI",
    "research": "",
    "draft": "",
    "critique": "",
    "final_article": "",
    "revision_count": 0,
    "needs_revision": False
}

# Run the graph
final_state = app.invoke(initial_state)

print("\n" + "=" * 60)
print("FINAL ARTICLE:")
print("=" * 60)
print(final_state["final_article"])
print(f"\nRevisions: {final_state['revision_count']}")

## 8. Debate System

The debate pattern is particularly useful for:
- Reducing bias in LLM outputs
- Exploring both sides of complex decisions
- Stress-testing ideas before committing to them

### Exercise 3: Build a Debate System

Build a multi-agent debate system with:
- **Pro Agent**: argues in favor of the proposition
- **Con Agent**: argues against the proposition
- **Judge Agent**: evaluates both sides and renders a verdict

**Requirements:**
- `debate(topic, rounds=3)` function
- Each round: pro argues, con responds, both see previous arguments
- After all rounds, the judge evaluates and decides
- Return the full debate transcript and verdict

In [ ]:
# Exercise 3: Multi-agent debate system

def debate(topic, rounds=3):
    """
    Run a multi-round debate between a pro agent and a con agent,
    with a judge agent rendering a final verdict.
    
    Args:
        topic: The proposition to debate
        rounds: Number of debate rounds
    Returns:
        dict with 'transcript' (list of dicts) and 'verdict' (string)
    """
    print(f"=== DEBATE ===")
    print(f"Topic: {topic}")
    print(f"Rounds: {rounds}")
    print("=" * 60)
    
    transcript = []
    
    for round_num in range(1, rounds + 1):
        # TODO: Build context from previous rounds for both agents
        context = None
        
        # TODO: Pro agent makes their argument
        # Should see previous arguments to build upon and counter
        pro_argument = None
        
        # TODO: Con agent responds
        # Should see pro's argument and previous context
        con_argument = None
        
        # TODO: Append to transcript
        pass
    
    # TODO: Judge evaluates the full debate and renders verdict
    verdict = None
    
    return {"transcript": transcript, "verdict": verdict}

# Test (will fail until you complete the TODOs)
# result = debate("AI-generated code will replace most human programmers within 10 years")

### Solution

In [ ]:
# Solution: Multi-agent debate system

def debate(topic, rounds=3):
    """
    Run a multi-round debate between a pro agent and a con agent,
    with a judge agent rendering a final verdict.
    """
    print(f"=== DEBATE ===")
    print(f"Proposition: {topic}")
    print(f"Rounds: {rounds}")
    print("=" * 60)
    
    transcript = []
    
    for round_num in range(1, rounds + 1):
        print(f"\n--- Round {round_num} ---")
        
        # Build context from previous rounds
        context = ""
        if transcript:
            context = "Previous rounds:\n"
            for entry in transcript:
                context += f"\nRound {entry['round']}:\n"
                context += f"PRO: {entry['pro']}\n"
                context += f"CON: {entry['con']}\n"
            context += "\n"
        
        # Pro agent argues
        print("\n[PRO Agent]")
        pro_prompt = f"{context}" if context else ""
        pro_prompt += f"This is round {round_num} of {rounds}. Make your strongest argument FOR the proposition."
        if round_num > 1:
            pro_prompt += " Address the opponent's previous points and strengthen your position."
        
        pro_argument = chat([
            {"role": "system", "content": f"""You are a skilled debater arguing IN FAVOR of this proposition: '{topic}'.
Make compelling, well-reasoned arguments. Use evidence and logic.
Keep your argument to 2-3 paragraphs."""},
            {"role": "user", "content": pro_prompt}
        ], max_tokens=400)
        print(pro_argument)
        
        # Con agent responds
        print("\n[CON Agent]")
        con_prompt = f"{context}" if context else ""
        con_prompt += f"\nPro's argument this round:\n{pro_argument}\n\n"
        con_prompt += f"This is round {round_num} of {rounds}. Counter the pro's arguments and make your strongest case AGAINST the proposition."
        
        con_argument = chat([
            {"role": "system", "content": f"""You are a skilled debater arguing AGAINST this proposition: '{topic}'.
Make compelling, well-reasoned counter-arguments. Use evidence and logic.
Keep your argument to 2-3 paragraphs."""},
            {"role": "user", "content": con_prompt}
        ], max_tokens=400)
        print(con_argument)
        
        transcript.append({
            "round": round_num,
            "pro": pro_argument,
            "con": con_argument
        })
    
    # Judge evaluates
    print(f"\n{'=' * 60}")
    print("[JUDGE]")
    
    full_debate = "\n\n".join([
        f"--- Round {entry['round']} ---\nPRO: {entry['pro']}\nCON: {entry['con']}"
        for entry in transcript
    ])
    
    verdict = chat([
        {"role": "system", "content": """You are an impartial debate judge. Evaluate the debate objectively.
Your verdict must include:
1. Summary of the strongest argument from each side
2. Which side argued more effectively and why
3. Your final verdict: PRO wins, CON wins, or DRAW
4. A brief explanation of your reasoning
Be fair and thorough."""},
        {"role": "user", "content": f"Proposition: {topic}\n\nFull debate transcript:\n\n{full_debate}"}
    ], max_tokens=600)
    
    print(verdict)
    
    return {"transcript": transcript, "verdict": verdict}

# Run the debate
debate_result = debate("AI-generated code will replace most human programmers within 10 years", rounds=3)

## 9. Refactoring the Trading Agent with LangGraph

The existing `ai_trading_agent.ipynb` uses LangChain's `SequentialChain` to connect 5 specialized agents:

```
Current Architecture (SequentialChain):

  ticker -> [Market Data] -> [Sentiment] -> [Macro] -> [Strategy] -> [Risk]
                                                            |
                                                         (linear,
                                                          no loops,
                                                          no conditionals)
```

With LangGraph, we could add much more sophisticated behavior:

```
Modernized Architecture (LangGraph StateGraph):

  ticker -> [Market Data] -+-> [Strategy]
            [Sentiment]  --+       |
            [Macro]      --+   (conditional)
            (parallel)         /         \
                         [Risk OK]    [Risk Too High]
                            |              |
                          Output     [Revise Strategy]
                                          |
                                     (loop back to Risk)
```

### Key improvements with LangGraph:

| Feature | SequentialChain | LangGraph |
|---------|----------------|----------|
| Execution order | Strictly linear | Flexible (parallel, conditional) |
| Error handling | Fails the entire chain | Can retry individual nodes |
| Conditional logic | Not supported | Built-in conditional edges |
| Looping | Not supported | Nodes can loop back |
| State management | Output variables only | Rich typed state |
| Observability | `verbose=True` | Full state inspection at each node |

### Conceptual Mapping

```python
# OLD: SequentialChain
sequential_agent = SequentialChain(
    chains=[market_data_chain, sentiment_chain, ...],
    input_variables=["ticker"],
    output_variables=["market_data", "sentiment_analysis", ...]
)

# NEW: LangGraph StateGraph
class TradingState(TypedDict):
    ticker: str
    market_data: str
    sentiment_analysis: str
    macro_analysis: str
    strategy: str
    risk_assessment: str
    risk_acceptable: bool

workflow = StateGraph(TradingState)
workflow.add_node("market_data", market_data_node)
workflow.add_node("sentiment", sentiment_node)
workflow.add_node("macro", macro_node)
workflow.add_node("strategy", strategy_node)
workflow.add_node("risk", risk_node)

# Can now add conditional edges, parallel execution, loops
workflow.add_conditional_edges("risk", check_risk, {...})
```

The key insight: LangGraph turns a rigid pipeline into a **flexible, observable state machine** that can handle real-world complexity like retries, conditionals, and feedback loops.

### Exercise 4: Multi-Agent Pipeline with LangGraph

Build a 3-agent content pipeline using LangGraph:

1. **Researcher**: gathers key facts about the topic
2. **Writer**: produces a draft article from the research
3. **Editor**: reviews and polishes the final output

**Requirements:**
- Define a `PipelineState` TypedDict with appropriate fields
- Create node functions for each agent
- Build a StateGraph with proper edges
- Add a conditional edge: if the editor's quality score is below 7, loop back to the writer
- Limit revisions to 2 maximum

In [ ]:
# Exercise 4: Multi-agent pipeline with LangGraph

# TODO: Define the state
class PipelineState(TypedDict):
    topic: str
    research: str
    draft: str
    editor_feedback: str
    final_output: str
    quality_score: int
    revision_count: int

# TODO: Implement researcher node
def research_node(state: PipelineState) -> dict:
    """Gather key facts about the topic."""
    return {"research": None}  # TODO

# TODO: Implement writer node
def write_node(state: PipelineState) -> dict:
    """Write or revise the article."""
    return {"draft": None, "revision_count": None}  # TODO

# TODO: Implement editor node
def edit_node(state: PipelineState) -> dict:
    """Review the article and provide a quality score."""
    return {"editor_feedback": None, "quality_score": None}  # TODO

# TODO: Implement output node
def output_node(state: PipelineState) -> dict:
    """Finalize the output."""
    return {"final_output": None}  # TODO

# TODO: Implement routing function
def route_after_edit(state: PipelineState) -> str:
    """Route to writer for revision or to output."""
    return None  # TODO: return "writer" or "output"

# TODO: Build and compile the StateGraph
# pipeline = StateGraph(PipelineState)
# ... add nodes, edges, conditional edges ...
# app = pipeline.compile()

# TODO: Run the pipeline
# result = app.invoke({"topic": "...", ...})

### Solution

In [ ]:
# Solution: Multi-agent pipeline with LangGraph

class PipelineState(TypedDict):
    topic: str
    research: str
    draft: str
    editor_feedback: str
    final_output: str
    quality_score: int
    revision_count: int


def research_node(state: PipelineState) -> dict:
    """Gather key facts about the topic."""
    print("[Researcher Node]")
    research = chat([
        {"role": "system", "content": "You are a thorough researcher. Provide 5 key facts and insights about the given topic. Include specific details, numbers, and examples where possible."},
        {"role": "user", "content": f"Research this topic thoroughly: {state['topic']}"}
    ], max_tokens=500)
    print(f"  Research gathered ({len(research)} chars)")
    return {"research": research}


def write_node(state: PipelineState) -> dict:
    """Write or revise the article."""
    print("[Writer Node]")
    revision_count = state.get("revision_count", 0) + 1
    
    if state.get("editor_feedback") and state.get("draft"):
        # Revision mode
        prompt = f"""Revise the article based on editor feedback.

Current draft:
{state['draft']}

Editor feedback:
{state['editor_feedback']}

Write an improved version that addresses all the feedback."""
    else:
        # Initial draft
        prompt = f"""Write a well-structured article (3-4 paragraphs) on the topic '{state['topic']}' 
using this research:

{state['research']}

Make it informative, engaging, and well-organized."""
    
    draft = chat([
        {"role": "system", "content": "You are an excellent writer. Write clear, engaging, well-structured content."},
        {"role": "user", "content": prompt}
    ], max_tokens=700)
    
    print(f"  Draft written (revision {revision_count}, {len(draft)} chars)")
    return {"draft": draft, "revision_count": revision_count}


def edit_node(state: PipelineState) -> dict:
    """Review the article and provide a quality score."""
    print("[Editor Node]")
    feedback = chat([
        {"role": "system", "content": """You are a senior editor. Review the article and:
1. Start with 'SCORE: X/10' (be honest and strict)
2. List specific strengths (2-3 points)
3. List specific areas for improvement (2-3 points)
Score 7+ means publishable quality."""},
        {"role": "user", "content": f"Review this article:\n\n{state['draft']}"}
    ], max_tokens=400)
    
    # Parse score
    score_match = re.search(r'SCORE:\s*(\d+)', feedback)
    score = int(score_match.group(1)) if score_match else 5
    
    print(f"  Quality score: {score}/10")
    return {"editor_feedback": feedback, "quality_score": score}


def output_node(state: PipelineState) -> dict:
    """Finalize the output."""
    print("[Output Node]")
    print(f"  Finalized after {state.get('revision_count', 0)} revision(s), score: {state.get('quality_score', 'N/A')}/10")
    return {"final_output": state["draft"]}


def route_after_edit(state: PipelineState) -> str:
    """Route to writer for revision or to output based on quality score."""
    score = state.get("quality_score", 0)
    revisions = state.get("revision_count", 0)
    
    if score < 7 and revisions < 3:
        print(f"  -> Routing back to writer (score {score} < 7, revision {revisions} < 3)")
        return "writer"
    else:
        reason = f"score {score} >= 7" if score >= 7 else f"max revisions reached ({revisions})"
        print(f"  -> Routing to output ({reason})")
        return "output"


# Build the graph
pipeline = StateGraph(PipelineState)

# Add nodes
pipeline.add_node("researcher", research_node)
pipeline.add_node("writer", write_node)
pipeline.add_node("editor", edit_node)
pipeline.add_node("output", output_node)

# Add edges
pipeline.add_edge(START, "researcher")
pipeline.add_edge("researcher", "writer")
pipeline.add_edge("writer", "editor")

# Conditional edge from editor
pipeline.add_conditional_edges(
    "editor",
    route_after_edit,
    {
        "writer": "writer",
        "output": "output"
    }
)

pipeline.add_edge("output", END)

# Compile
content_pipeline = pipeline.compile()

print("Content pipeline compiled!")
print("")
print("Graph:")
print("  START -> researcher -> writer -> editor")
print("                           ^          |")
print("                           |     (score < 7?)")
print("                           +--- yes ---+")
print("                                  |")
print("                                  no")
print("                                  |")
print("                              output -> END")

In [ ]:
# Run the pipeline
print("=== Running Content Pipeline ===")
print("=" * 60)

result = content_pipeline.invoke({
    "topic": "How large language models are transforming software development",
    "research": "",
    "draft": "",
    "editor_feedback": "",
    "final_output": "",
    "quality_score": 0,
    "revision_count": 0
})

print("\n" + "=" * 60)
print("FINAL OUTPUT:")
print("=" * 60)
print(result["final_output"])
print(f"\nTotal revisions: {result['revision_count']}")
print(f"Final quality score: {result['quality_score']}/10")

## 10. Summary & Key Takeaways

### What we covered

1. **Multi-Agent Patterns**: Four fundamental architectures -- sequential, parallel, hierarchical, and debate -- each suited to different problem types.

2. **ReAct from Scratch**: Implemented the full Thought-Action-Observation loop without any framework, showing how agents reason about tool usage step by step.

3. **Agent Memory**: Built buffer and summary memory systems that allow agents to maintain context across multiple conversation turns.

4. **Agent Planning**: Decomposing complex goals into sub-tasks, executing them sequentially with cumulative context, and synthesizing the results.

5. **Self-Reflection**: The generate-critique-revise pattern that allows agents to iteratively improve their own output.

6. **LangGraph**: Building agent workflows as state machines with nodes, edges, conditional routing, and feedback loops.

7. **Debate Systems**: Multi-agent debate with pro/con agents and an impartial judge, useful for balanced analysis and bias reduction.

8. **Trading Agent Modernization**: How the existing `ai_trading_agent.ipynb` could be refactored from SequentialChain to a more flexible LangGraph StateGraph.

### Key Design Principles

| Principle | Description |
|-----------|-------------|
| **Single Responsibility** | Each agent should have one clear role |
| **Explicit State** | Use typed state objects to pass data between agents |
| **Observability** | Log each agent's input/output for debugging |
| **Graceful Degradation** | Set max retries/iterations to prevent infinite loops |
| **Composability** | Design agents that can be reused in different workflows |

### Looking Ahead

Multi-agent systems are rapidly evolving. Key areas to watch:
- **Agent-to-agent communication protocols** (e.g., AutoGen, CrewAI)
- **Persistent agent memory** with vector databases
- **Autonomous agent loops** with human-in-the-loop checkpoints
- **Tool-use standardization** (e.g., OpenAI function calling, Anthropic tool use)

---

### References

- **Paper**: Park et al. *"Generative Agents: Interactive Simulacra of Human Behavior"* (2023) -- Simulated town of 25 AI agents with memory, planning, and reflection
- **Paper**: Shinn et al. *"Reflexion: Language Agents with Verbal Reinforcement Learning"* (2023) -- Self-reflection pattern for agent improvement
- **Paper**: Yao et al. *"ReAct: Synergizing Reasoning and Acting in Language Models"* (2022) -- The foundational ReAct framework
- **Documentation**: [LangGraph Documentation](https://langchain-ai.github.io/langgraph/) -- Official guide for building agent workflows as state machines
- **Course**: DeepLearning.AI *"AI Agents in LangGraph"* -- Practical course on building production agent systems